$\newcommand{\cusps}{\operatorname{cusps}}$
$\newcommand{\im}{\operatorname{im}}$
$\newcommand{\Xsplit}{X_{\text{sp}}}$
$\newcommand{\Xnonsplit}{X_{\text{ns}}}$
$\newcommand{\XA}{X_{A_4}}$
$\newcommand{\mat}[4]{\left(\begin{matrix} #1 & #2 \\ #3 & #4  \end{matrix}\right)}$
$\def\Z{{\mathbb Z}}$
$\def\Q{{\mathbb Q}}$
$\def\R{{\mathbb R}}$
$\def\C{{\mathbb C}}$
$\def\T{{\mathbb T}}$
$\def\H{{\mathbb H}}$
$\def\P{{\mathbb P}}$   
$\newcommand{\GL}{\operatorname{GL}}$
$\newcommand{\SL}{\operatorname{SL}}$
$\newcommand{\PGL}{\operatorname{PGL}}$
$\newcommand{\PSL}{\operatorname{PSL}}$

# Computing the  modular curves $\Xsplit(13)$, $\Xnonsplit(13)$ and $\XA(13)$ using modular symbols in Sage

## Barinder Banwait and John Cremona

*This notebook consists of annotated Sage code which computes
the three genus $3$ modular curves $\Xsplit(13)$, $\Xnonsplit(13)$ and
$\XA(13)$ of level $13$, together with the $j$-map for each and the images of the (small) rational points.*

## Preliminaries: defining the congruence subgroup and modular  symbol space

We set the level and define the congruence
subgroup $G=\tilde{\Gamma}(13)=\Gamma_0(13^2)\cap\Gamma_1(13)$, which
is conjugate to $\Gamma(13)$:

In [1]:
N = 13
G = GammaH(N^2,[N+1])

Next we define the associated modular symbol space $MG$ (with
sign $0$), and its cuspidal subspace $C$ of dimension $d$.

In [2]:
M = ModularSymbols(G,sign=0)
C = M.cuspidal_subspace()
print("C = {}".format(C))
d = C.dimension()
print("\nC has dimension {} and M has dimension {}".format(d, M.dimension()))

C = Modular Symbols subspace of dimension 100 of Modular Symbols space of dimension 183 for Congruence Subgroup Gamma_H(169) with H generated by [14] of weight 2 with sign 0 and over Rational Field

C has dimension 100 and M has dimension 183


The dimension of $C$ is twice the genus of
the associated modular curve $\tilde{X}(13)$.  

In [3]:
G.genus()

50

For future reference we
will need the basis for $C$ in terms of the ambient modular symbols
space $M$, and the
matrix defining $C$ as a subspace of $M$.

In [4]:
Cb=C.basis()
Cmat = Matrix([c.list() for c in Cb]).transpose()

Next we decompose the cuspidal space into old and new subspaces, then
decompose the new part into simple components with respect to the
Hecke action:

In [5]:
Cold = C.old_subspace()
Cnew = C.new_subspace()
Cnew_dec = Cnew.decomposition()

The number of simple new summands and their
dimensions are 

In [6]:
print(len(Cnew_dec))
print([c.dimension() for c in Cnew_dec])

10
[4, 4, 4, 6, 6, 8, 12, 12, 12, 24]


*Remark on row and column vectors and matrices: I think of
elements of the modular symbol space (specifically $C$) as represented
by column vectors (of length $10$ for $C$), and so the matrices
which represent operators act by multiplying these vectors on the
left.  We construct $T$ and ${\tt Conj}$ like this below.  However,
Sage's default is that matrices act by right multiplication of row
vectors.  So when we compute $W_{13}$ using Sage's ${\tt atkin\_lehner}$
function we have to transpose it to be compatible with the $T$ we
compute "manually".  Later, we form various subspaces of
$\Q^{100}$ (and $F^{100}$ for various fields $F$), which are
always given by a basis matrix of size $d\times100$ (where $d$ is the
dimension of the subspace) whose rows are an echelon basis.*

*Also, Sage makes no distinction between row and column vectors.*

$\DeclareMathOperator{\PSL}{PSL}$
## Computation of Hecke and other operators to get the  representation of $\PSL(2,13)$ on the modular symbol space


Computation of $\tilde{T}=\mat{1}{1/{13}}{0}{1}$:  the
images of the generating symbols are:

In [7]:
T_ims =  [ C(sum([ s[0] * sum([ mm[0]*mm[1].manin_symbol_rep()
           for mm in s[1].apply([1,1/N,0,1])])
           for s in c.modular_symbol_rep()]))
           for c in Cb]

We use these to construct the matrix:

In [8]:
T = Matrix([(Cmat\vector(im.list())).list() for im in T_ims]).transpose()

and check that it does what it should:

In [9]:
assert all([T_ims[j] == sum([T[i,j]*Cb[i] for i in range(d)])
           for j in range(d)]), "action of T is incorrect"

Next we compute the matrix of complex conjugation:

In [10]:
Conj = C.star_involution().matrix().transpose()

Next we compute the matrix of
$\tilde{S}=\mat{0}{-1}{121}{0}$, the Atkin-Lehner
involution. Note that the built-in operators (including $T_2$ below)
need to be transposed before they act on the left on column-vectors,
since they are designed to act on the right on row-vectors.  From now
on we will use the notation $S$, $T$ instead of $\tilde{S}$,
$\tilde{T}$. The Sage identifiers ${\tt S}$, ${\tt T}$ refer to the
$100\times100$ matrices giving the action of these on $C$, identified
with $\Q^{100}$. 

In [11]:
W13 = C.atkin_lehner_operator()
S = W13.matrix().transpose()

Again we check that the action is as expected:

In [12]:
assert all([W13(Cb[j]) == sum([S[i,j]*Cb[i] for i in range(d)])
       for j in range(d)]), "action of S is incorrect"

As a consistency check we check that that all these $100\times100$
matrices have the right properties: they are the images of the
standard generators of $\PSL(2,\Z)$ under the $100$-dimensional
represntation on the cuspidal modular symbol space, which factors
through $\PSL(2,13)$.

In [13]:
I = idmat =  S.parent()(1)
assert S^2 == T^N == (T*S)^3 == Conj^2 == I

Note that conjugating with ${\tt Conj}$ inverts $T$ and fixes $S$.

In [14]:
assert Conj*T*Conj*T==I
assert Conj*S*Conj==S

We are interested in the subspace fixed by a subgroup of $\PSL(2,13)$
isomorphic to $A_4$.  Such a subgroup is defined by taking two
generating matrices in $\PSL(2,13)$; we choose one such subgroup
(among possible conjugates) to contain $S=\mat{-1}{0}{0}{1}$,
so that the corresponding congruence subgroup will be of real type.  We
lift the generators to $\PSL(2,\Z)$, express them as words in the usual
generators $S$, $T$ of $\PSL(2,\Z)$, and then take the same words in
our $100\times100$ matrices $S$,$T$.

In [15]:
SZ = matrix(ZZ,2,2,[0,-1,1,0])
TZ = matrix(ZZ,2,2,[1,1,0,1])

The first generator $A$ is $\mat{-5}{0}{0}{5}\in
\PSL(2,13)$ which lifts to the element
$T^5ST^{-2}ST^2ST^3ST^{-5}=$

In [16]:
AZ = TZ^5*SZ*TZ^(-2)*SZ*TZ^2*SZ*TZ^3*SZ*TZ^(-5)
print(AZ)
assert AZ^2 == -1
assert AZ%13 == matrix([[-5,0],[0,5]])%13

[-70 377]
[-13  70]


of order $2$ in $\PSL(2,\Z)$.  Hence we set

In [17]:
A = T^5*S*T^(-2)*S*T^2*S*T^3*S*T^(-5)

The second generator $B$ is $\mat{2}{2}{3}{-3} \in
\PSL(2,13)$ which lifts to the element
$T^4ST^3ST^{-3}S=$

In [18]:
BZ = TZ^4*SZ*TZ^3*SZ*TZ^(-3)*SZ
print(BZ)
assert BZ%13 == matrix([[2,2],[3,-3]])%13

[-37 -11]
[-10  -3]


of order $3$ in $\PSL(2,\Z)$.

In [19]:
B = T^4*S*T^3*S*T^(-3)*S

We now verify that the group $A_4$ generated by $A$ and $B$ is
normalised by conjugation, and that various other identities hold.
Here ${\tt  Ad}=A'=B^{-1}AB\equiv\mat{0}{1}{1}{0}\mod{13}$, so
that $A,A'$ commute and generate the Klein $4$-subgroup of $A_4$ whose
third element is $AA'=A'A=\mat{0}{1}{-1}{0}=S$.

In [20]:
assert Conj*A*Conj==A
assert Conj*B*Conj == A*B*A
Ad = B^(-1)*A*B # = [0,1;1,0] (mod 13)
assert A*Ad==Ad*A # = [0,1;-1,0] (mod 13)
assert A*Ad==S

To obtain the $A_4$-invariant subspace we construct the associated
idempotent (scaled by the group order $12$).  Now the subspace of
$A_4$-invariant vectors is spanned by the columns of ${\tt sigma}$.

In [21]:
sigma = (I+A)*(I+Ad)*(I+B+B^2)
assert sigma.rank() == 6
assert (sigma-12).rank() == d-6
assert sigma*(sigma-12)==0 # idempotency (scaled)

A4_inv = sigma.column_space()
A4_inv_bas = A4_inv.basis_matrix().transpose()
A4_inv.dimension()

6

The (doubled) dimension of this $A_4$-invariant subspace is
$6$.   We wish to know which of the old and
new simple components this intersects.  To do this we project onto
each of them using the dual basis.  First, the old space contains
nothing $A_4$-invariant:

In [22]:
oldproj = Cold.dual_free_module().basis_matrix()*Cmat
assert oldproj*(A4_inv.basis_matrix().transpose()) == 0

Next we see that two of the new components will be involved:

In [23]:
newproj = [comp.dual_free_module().basis_matrix()*Cmat for comp in Cnew_dec]
new_A4_components = [i for i in range(len(Cnew_dec))
       if newproj[i]*(A4_inv.basis_matrix().transpose()) != 0]
print("Relevant new components have indices {} and dimensions {}"
      .format(new_A4_components, [Cnew_dec[comp].dimension() for comp in new_A4_components]))

Relevant new components have indices [3, 8] and dimensions [6, 12]


In order to compute the equation of the modular curve we could work
exclusively with the sum of these two new components, which will be
denoted ${\tt S3}$ and ${\tt S8}$ below.  The first of these has trivial
character and Hecke eigenvalues in the cubic field $\Q(\zeta_7^+)$,
while the second is the sum of its twists by the order $3$ character
modulo $13$ (denoted ${\tt chi}$ later).  These two twists are combined
here as the decomposition has been carried out over $\Q$.  But it will
be more convenient (specifically when computing the coordinates of the
seven cusps) to work in a larger space which is $\PSL(2,\Z)$-invariant: this
is the sum of all the twists of ${\tt S3}$ under all the characters
modulo $13$, which form a cyclic group of order $12$ with generator
denoted ${\tt eps}$.

In [24]:
S3=Cnew_dec[3]
S8=Cnew_dec[8]
comps = [3,4,6,7,8,9]

These components have the following (doubled) dimensions:

In [25]:
[Cnew_dec[c].dimension() for c in comps]

[6, 6, 12, 12, 12, 24]

Identifying the cuspidal modular symbol space ${\tt C}$ with $\Q^{100}$,
we denote by ${\tt V3}$ the subspace of $\Q^{100}$ corresponding to ${\tt
  S3}$, and by ${\tt W}$ the $\PSL(2,\Z)$-invariant sum:

In [26]:
Q100 = QQ^100
V3 = Q100.subspace([C.free_module().coordinates(v)
                    for v in S3.free_module().basis()])
W = Q100.subspace(sum([[C.free_module().coordinates(v)
                        for v in Cnew_dec[c].free_module().basis()]
                        for c in comps],[]))

These have (doubled) dimensions

In [27]:
V3.dimension()

6

and

In [28]:
W.dimension()

72

respectively.  Note that these subspaces of
$\Q^{100}$ are "row subspaces'', spanned by the rows of their basis
matrices, while our $100\times100$ matrices ${\tt S,T,A0,B0,Conj}$ act
on the left on column vectors.  This explains the transposing which
goes on in what follows.

We compute the Hecke operator $T_2$ (which is already transposed) and
its resrictions to the two subspaces:

In [29]:
V3plus = (Conj-1).transpose().kernel_on(V3)
Wplus =  (Conj-1).transpose().kernel_on(W)
V3plus.dimension(), Wplus.dimension()

(3, 36)

These have half the dimensions of ${\tt V3}$ and ${\tt W}$.  

We compute the Hecke operator $T_2$ (which is already transposed) and
its resrictions to the two subspaces:

In [30]:
T2 = C.hecke_matrix(2)
T2V3plus = T2.restrict(V3plus)
T2Wplus = T2.restrict(Wplus)

## Definition of various number fields

Now we define various cyclotomic fields, all contained in
$\Q(\zeta_{1092})$ ---note that 1092 =

In [31]:
 1092.factor() 

2^2 * 3 * 7 * 13

---with embeddings between them and some automorphisms:

In [32]:
Q1092.<zeta1092> = CyclotomicField(1092)
conj1092 = Q1092.hom([zeta1092^(-1)])  # complex conjugation on Q1092
#
Q13.<zeta13> = CyclotomicField(13, embedding=zeta1092^84)
eQ13Q1092 = Q13.hom([Q1092(zeta13)])   # embedding Q13 into Q1092
Q13sigma = Q13.hom([zeta13^4])         # auto of order 3 of Q13
Q13aut = Q13.hom([zeta13^2])           # auto of order 6 of Q13
#
Q91.<zeta91> = CyclotomicField(91,embedding=zeta1092^12)
eQ91Q1092 = Q91.hom([zeta1092^12])    # embedding Q91 into Q1092
eQ13Q91 = Q13.hom([zeta91^7])         # embedding Q13 into Q91
Q91aut = Q91.hom([zeta91^53])         # auto of order 3 of Q91 fixing Q13
#
Q84.<zeta84> = CyclotomicField(84, embedding=zeta1092^13)
conj84 = Q84.hom([zeta84^(-1)])       # complex conjugation on Q84
Q12.<zeta12> = CyclotomicField(12, embedding=zeta84^7)
Q7.<zeta7> = CyclotomicField(7, embedding=zeta84^12)
# override pretty-print for these fields:
Q1092._latex_ = lambda: "\\mathbb{Q}(\\zeta_{1092})"
Q13._latex_ = lambda: "\\mathbb{Q}(\\zeta_{13})"
Q91._latex_ = lambda: "\\mathbb{Q}(\\zeta_{91})"
Q84._latex_ = lambda: "\\mathbb{Q}(\\zeta_{84})"
Q12._latex_ = lambda: "\\mathbb{Q}(\\zeta_{12})"
Q7._latex_ = lambda: "\\mathbb{Q}(\\zeta_{7})"

$\newcommand{\Qzeta}[1]{\mathbb{Q}(\zeta_{#1})}$
$\newcommand{\Qzetaplus}[1]{\mathbb{Q}(\zeta_{#1}^+)}$
$\newcommand{\Qzetaplusplus}[1]{\mathbb{Q}(\zeta_{#1}^{++})}$
$\newcommand{\Qzetac}[1]{\mathbb{Q}(\zeta_{#1}^{c})}$
Inside $\Qzeta7$ we define the cubic real subfield $\Qzetaplus7$,
and inside $\Qzeta{13}$ the cubic subfield $\Qzetaplusplus{13})$ and
  the quartic subfield $\Qzetac{13}$:

In [33]:
zeta7p=zeta7+1/zeta7
Q7p.<zeta7p>=NumberField(zeta7p.minpoly(), embedding=zeta7+1/zeta7)
#
zeta13pp = zeta13+zeta13^5+zeta13^8+zeta13^12 # generates cubic subfield
Q13pp.<zeta13pp> = NumberField(zeta13pp.minpoly(), embedding=zeta13pp)
eQ13ppQ13 = Q13pp.hom([Q13(zeta13pp)])
#
zeta13c = zeta13+zeta13^3+zeta13^9 # generates quartic subfield
Q13c.<zeta13c> = NumberField(zeta13c.minpoly(), embedding=zeta13c)
eQ13cQ13 = Q13c.hom([Q13(zeta13c)])
#
Q7p._latex_ = lambda: "\\mathbb{Q}(\\zeta_{7}^{+})"
Q13pp._latex_ = lambda: "\\mathbb{Q}(\\zeta_{13}^{++})"
Q13c._latex_ = lambda: "\\mathbb{Q}(\\zeta_{13}^{c})"

For computations with $q$-expansions we define some power series
rings:

In [34]:
ps_prec = 200
Q1092q.<q> = PowerSeriesRing(Q1092,ps_prec)
Q91q.<q> = PowerSeriesRing(Q91,ps_prec)
Q13q.<q> = PowerSeriesRing(Q13,ps_prec)
QQq.<q> = PowerSeriesRing(QQ,ps_prec)
#
ps_hiprec = 500 # 5000
Q13ppq.<q> = PowerSeriesRing(Q13pp,ps_prec)
Q1092qhi.<q> = PowerSeriesRing(Q1092,ps_hiprec)
Q91qhi.<q> = PowerSeriesRing(Q91,ps_hiprec)
Q13qhi.<q> = PowerSeriesRing(Q13,ps_hiprec)
QQqhi.<q> = PowerSeriesRing(QQ,ps_hiprec)

## Hecke eigenvalues and eigenvectors, and twisting operators

Now we start to compute Hecke eigenvalues (of $T_2$) and associated
eigenvectors.  The eigenvalues of $T_2$ on ${\tt V3}$ are in
$\Qzetaplus7$ (this will be verified later).  We define the first of
these to be $a$, then find its three Galois conjugates, and reorder them to
make sure that $a$ is first in the list.  The list of $3$ conjugate
eigenvalues is called ${\tt eig3}$:

In [35]:
a=1/zeta7p
eig3 = a.galois_conjugates(Q7p) # this does not put a first
eig3.remove(a)
eig3=[a]+eig3
assert Set(eig3) == Set(T2V3plus.change_ring(Q7p).eigenvalues())

To get all $36$ eigenvalues of $T_2$ on ${\tt Wplus}$ we multiply these
by the $12$th roots of unity:

In [36]:
eig12 = [a*zeta12^j for j in range(12)]
eig36 = sum([[Q84(ai)*zeta12^j for j in range(12)] for ai in eig3],[])
eig9 = sum([[eig36[12*i+4*j] for i in range(3)] for j in range(3)],[])
assert all([T2Wplus.charpoly()(l)==0 for l in eig36])

Here $\tt eig12$ = 

In [37]:
eig12

[-zeta84^22 + zeta84^8 + zeta84^6 - 1,
 zeta84^13 - zeta84^7 + zeta84,
 zeta84^20 - zeta84^14 + zeta84^8,
 zeta84^23 - zeta84^17 + zeta84^13 + zeta84^11 - zeta84^7 - zeta84^5 + zeta84,
 zeta84^22 + zeta84^20 - zeta84^14 - zeta84^6 + 1,
 zeta84^23 - zeta84^17 + zeta84^11 - zeta84^5,
 zeta84^22 - zeta84^8 - zeta84^6 + 1,
 -zeta84^13 + zeta84^7 - zeta84,
 -zeta84^20 + zeta84^14 - zeta84^8,
 -zeta84^23 + zeta84^17 - zeta84^13 - zeta84^11 + zeta84^7 + zeta84^5 - zeta84,
 -zeta84^22 - zeta84^20 + zeta84^14 + zeta84^6 - 1,
 -zeta84^23 + zeta84^17 - zeta84^11 + zeta84^5]

are the eigenvalues of $T_2$ on the $12$-dimensional space spanned
by the newform with $a_2=a$ and its twists; this $12$-dimensional
space is an irredicible $\PSL(2,13)$-module over $\Qzetaplus7$.  The
longer list ${\tt eig36}$ includes all Galois conjugates over $\Q$, and
is the list of all $36$ eigenvalues of $T_2$ on
${\tt Wplus}$.  Lastly, ${\tt eig9}$ is a list of the $T_2$-eigenvalues on
the space $V_3$ and its Galois conjugates.

In [38]:
Wplus.dimension()

36

Now for the eigenvectors.  We compute the first three
$T_2$-eigenvectors with respect to the basis of ${\tt V3plus}$, and
check that their eigenvalues are the elements of ${\tt eig3}$ in the
correct order.  

In [39]:
evec3 = [(T2V3plus-e).kernel().basis()[0] for e in eig3]
assert all([v*T2V3plus==e*v for e,v in zip(eig3,evec3)])

The list of these eigenvectors is called ${\tt evec3}$; each has length $3$:

In [40]:
assert all (len(e)==3 for e in evec3)

 and entries in $\Qzetaplus7$:

In [41]:
assert all (e[0].parent() == Q7p for e in evec3)

Next, multiplying by the basis matrix for ${\tt V3}$ as a subspace of
$\Q^{100}$ we obtain the corresponding three eigenvectors for $T_2$.

In [42]:
longevec3 = [e*V3plus.basis_matrix() for e in evec3]
assert all([v*T2==ev*v for v,ev in zip(longevec3,eig3)])

The list of these $3$ eigenvectors is called ${\tt
  longevec3}$; each has length $100$ and entries in
$\Qzetaplus7$.

$\def\eps{\varepsilon}$
To compute the remaining eigenvectors which are an eigenbasis for the
$36$-dimensional space ${\tt Wplus}$, we apply the
twisting operator $R_{\eps}$ to these; this is quicker than calling
${\tt kernel()}$ many more times, and also ensures that the conjugate
eigenvectors are coherently scaled.

To do this we first need to define the characters modulo $13$.  The
character group is generated by ${\tt eps}=\eps$ of order $12$,
normalised so that $\eps(2)=\zeta_{12}$. We also use ${\tt
  chi}=\chi=\eps^4$ of order $3$.  We ensure that their values are in
the field $\Qzeta{12}\subset\Qzeta{1092}$ as constructed earlier.

In [43]:
eps = [eps for eps in DirichletGroup(13,base_ring=Q12)
           if eps(2)==zeta12][0]
chi = eps^4
assert chi(2) == zeta12^4
chibar = chi^2
epsbar = eps^(-1)

Next we compute the Gauss sum for $\eps$ and its conjugate, which lie
in $\Qzeta{84}$:

In [44]:
geps = sum([Q1092(zeta13)^i*eps(i) for i in range(13)])
gepsbar = sum([Q1092(zeta13)^i*epsbar(i) for i in range(13)])

The following vectors of scalings will be used to attach the correct
weights to eigenforms.  The point is that we will express
$A_4$-invariant eigenvectors in the space of cuspidal modular symbols
as linear combinations of the Hecke eigenvectors, and want to get hold
of the $q$-expansions of the corresponding cusp forms as linear
combinations of the (normalized) eigenforms; but there is not obvious
way of normalizing elements of the modular symbol space.  The
mathematics behind this scaling has been written up separately.

In [45]:
gepsvec = vector([geps^i for i in range(12)])
gepsvecm = vector([(-geps)^i for i in range(12)])

Now we use iteration to find $T^i$ for $0\le i<13$, as $100\times100$
matrices over $\Q$, then base-change these to $100\times100$ matrices
over $\Qzeta{12}$ so we can take linear combinations of them with
coefficients which are character values.

In [46]:
Tpowers=[I]
while len(Tpowers)<13: Tpowers+=[T*Tpowers[-1]]
TpowersQ12 = [M.change_ring(Q12) for M in Tpowers]

Using these powers of $T$, we form the twisting operators:

In [47]:
Repsbar = sum([eps(i)*TpowersQ12[i] for i in range(13)])
Reps = sum([epsbar(i)*TpowersQ12[i] for i in range(13)])
assert Conj*Reps==-Reps*Conj
assert T*Reps==Reps*T

In [48]:
Rchibar = sum([chi(i)*TpowersQ12[i] for i in range(13)])
Rchi = sum([chibar(i)*TpowersQ12[i] for i in range(13)])
assert Conj*Rchi==Rchi*Conj
assert T*Rchi==Rchi*T

Note that $T$ commutes with both ${\tt Reps}$ and ${\tt Rchi}$, but that
${\tt Conj}$ commutes with ${\tt Rchi}$ but anticommutes with ${\tt Reps}$.
This is because $\eps(-1)=-1$ while $\chi(-1)=+1$.

Now we compute the 12 eigenvectors as twists of the first one, each as
a list representing an elements of $\Qzeta{84}^{100}$:

In [49]:
longevec12=[(longevec3[0]).change_ring(Q84)]
R=Repsbar.change_ring(Q84)
while len(longevec12)<12: longevec12+=[R*longevec12[-1]]

If we twist one further time we will get a multiple of the original
vector, since $R_{\overline{\eps}}^{12}$ is a scalar multiple of the
identity.  For ease of exposition we have computed this scalar and define it here to be ${\tt r}$:

In [50]:
r = -1894464*zeta12^3 + 396825*zeta12^2 + 161460*zeta12 + 4259255
assert R*longevec12[11]==r*longevec12[0]
assert r.norm()==13^24

We now have $12$ eigenvectors in $\Qzeta{84}^{100}$ spanning a
$12$-dimensional space called ${\tt W1}$.  The $36$-dimensional space
${\tt Wplus}$ we had earlier is the sum of this and its (cubic) Galois
conjugates.

In [51]:
W1=longevec12[0].parent().subspace(longevec12)
Mlongevec12 = Matrix(longevec12)

The rows of the $12\times100$ matrix ${\tt Mlongevec2}$ are the
eigenvectors in ${\tt longevec12}$, which span ${\tt W1}$: note that
${\tt W1.basis\_matrix()}$ will not be equal to this as it has been
echelonised when the space ${\tt W1}$ was constructed.

${\tt W1}$ is an irreducible representation space for $\PSL(2,13)$,
which happens to have a $1$-dimensional $A_4$-invariant subspace.  It
is really defined over the cubic field $\Qzetaplus7$, but we have
extended scalars to $\Qzeta{84}$ which contains
$\Qzetaplus7(\zeta_{12})$.

On the $12$-dimensional space ${\tt W1}$, the action of $\PSL(2,13)$ is
via the images of $S$ and $T$, and $A_4$ acts as before using its generators
$A,B$:

In [52]:
TW1 = T.transpose().change_ring(Q84).restrict(W1)
SW1 = S.transpose().change_ring(Q84).restrict(W1)
AW1 = A.transpose().change_ring(Q84).restrict(W1)
BW1 = B.transpose().change_ring(Q84).restrict(W1)

In [53]:
AW1d = BW1^(-1)*AW1*BW1
sigmaW1 = (1+AW1)*(1+AW1d)*(1+BW1+BW1^2)
assert sigmaW1.rank()==1
assert sigmaW1*sigmaW1==12*sigmaW1
assert sigmaW1.row_space().dimension() == 1

As with ${\tt sigma}$ earlier, ${\tt sigmaW1}$ is the sum of the
$A_4$-action matrices, and projects ${\tt W1}$ onto the $1$-dimensional
$A_4$-invariant subspace.  (Strictly, ${\tt sigmaW1}$ is $12$ times the
projector.)

The $A_4$-invariant subspace of ${\tt W1}$ is $1$-dimensional, so we pick its
first basis vector and name it ${\tt Aw4\_inv\_W1}$, then check that it
really is $A_4$-invariant:

In [54]:
A4_inv_W1 = sigmaW1.row_space().basis()[0]
assert all([A4_inv_W1*g == A4_inv_W1 for g in [AW1,BW1]])

The coordinates of ${\tt A4\_inv\_W1}$ are with respect to the echelon
basis of ${\tt W1}$, but we want its coordinates with respect to our
original basis of Hecke eigenvectors (each the image of the previous
under the twist ${\tt Repsbar}$) which are the rows of ${\tt
  Mlongevec12}$.  We compute this by solving a linear system, giving
a new coordinate vector of the same length, called ${\tt A4\_inv\_1}$:

In [55]:
A4_inv_1 = Mlongevec12.solve_left(A4_inv_W1*W1.basis_matrix())

From earlier calculations we know that the only twists which appear
here, i.e. the only nonzero entries in the vector ${\tt A4\_inv\_1}$,
are those indexed by multiples of $4$, i.e. the twists by powers of
$\chi=\eps^4$:

In [56]:
assert all([A4_inv_1[i]==0 for i in range(12) if i%4!=0])

We pick out the nonzero coordinates of this $A_4$-invariant vector,
which give its coordinates with respect to the $1,\chi,\chi^2$ twists:

In [57]:
coeffs1=[A4_inv_1[4*i]  for i in range(3)]

We compute an automorphism of $\Qzeta{84}$ which fixes $\zeta_{12}$
and acts nontrivially on $\zeta_7^+$.  Those which fix $\zeta_{12}$
form a group of order $6$, of which $2$ fix $\zeta_7^+$, leaving $4$, with
only $2$ different restrictions to $\Qzetaplus7$, one the inverse of the other.
Some experimentation was needed to get the 3-cycle the right way round here
which turned to be with the automorphism indexed $1$:

In [58]:
aut7 = [aut for aut in Q84.automorphisms()
             if aut(zeta12)==zeta12 and aut(zeta7p)!=zeta7p][1]

We check that our automorphism called ${\tt aut7}$ cycles round the basic $3$
$T_2$-eigenvalues in the right direction:

In [59]:
assert all([aut7(eig3[i])==eig3[(i+1)%3] for i in range(3)])

Recall that ${\tt coeffs1}$ gives the $3$ coefficients of an
$A_4$-invariant vector in terms of the $1,\chi,\chi^2$ twists of the
original $T_2$-eigenvector.  To get the other two $A_4$-invariant
vectors we apply our automorphism to these:

In [60]:
coeffs2 = [aut7(c) for c in coeffs1]
coeffs3 = [aut7(c) for c in coeffs2]
coeffs = [coeffs1,coeffs2,coeffs3]

Each of these coefficient vectors gives the coefficients of an
$A_4$-invariant vector with respect to vectors in the modular symbol
space which are each Hecke eigenvectors, but whose relative scaling is
such that the twisting operator ${\tt Repsbar}$ maps them round
cyclically.  The theory of twisting operators (see Atkin and Li) tells
us that the corresponding $A_4$-invariant linear combinations of the
normalised newforms must use as coefficients not these coefficients
exactly, but these scaled by powers of the Gauss sum: this is
because the twisting operator takes a normalised eigenform to another
eigenform which is not normalised but scaled by a Gauss sum.  We have
already computed the required powers.  So each of the three elements
of the list ${\tt coeffs\_scaled}$ is a list of three coefficients such
that when we take these linear combinations of the newforms,  we will get
an $A_4$-invariant q-expansion.  [In fact they will also need to be
  complex-conjugated later.]

In [61]:
coeffs_scaled=[[c*geps^(4*i) for i,c in enumerate(co)] for co in coeffs]

## $q$-expansions of newforms and $A_4$-invariant cusp forms

Now we construct the $q$-expansion of a basis for the $A_4$-invariant
cusp forms, starting with the $q$-expansion of the newforms and taking
appropriate linear combinations.

The 9 relevant newforms have $q$-expansions which
are power series in $\Qzeta{1092}[[q]]$.  They actually have coefficients
in $\Q(\zeta_{12},\zeta_{7}^+)\subset\Qzeta{84}$, but we need to
extend scalars to include $\zeta_{13}$ so that we can take
linear combinations weighted by powers of Gauss sums, so we will
embed them in $\Qzeta{1092}[[q]]$.


In [62]:
f = S3.q_eigenform(ps_prec,'alpha')
g = S8.q_eigenform(ps_prec,'beta')

The coefficient field $\Q(f)=\Q(\alpha)=\Qzetaplus{7}$ is a subfield of
$\Qzeta{1092}$.  We order the embeddings $\Q(\alpha)
\hookrightarrow\Qzeta{1092}$  so that the
$i$'th one takes $\alpha$ to ${\tt eig3[i]}$ for $i=0,1,2$.

In [63]:
Kf=f.parent().base_ring()
alpha=Kf.gen()
embsKfQ84=Kf.embeddings(Q84)
emb3=[[e for e in embsKfQ84 if e(alpha)==ev][0] for ev in eig3]
assert all([e(alpha)==ev for e,ev in zip(emb3,eig3)])

Similarly, $\Q(g)=\Q(\beta)=\Q(\zeta_{7}^{+},\zeta_3)$ is also a
subfield of $\Qzeta{1092}$.  We order the
embeddings $\Q(\beta)
\hookrightarrow\Qzeta{1092}$  so that the $i$'th one takes $\beta$ to
${\tt eig9[i+3]}$  for $0\le i<6$.

In [64]:
Kg=g.parent().base_ring()
beta=Kg.gen()
embsKgQ84=Kg.embeddings(Q84)
emb8=[[e for e in embsKgQ84 if e(beta)==ev][0] for ev in eig9[3:]]
assert all([e(beta)==ev for e,ev in zip(emb8,eig9[3:])])

To get $q$-expansions for all $9$ newforms in $\Qzeta{1092}[[q]]$, we
apply the three embeddings of $\Q(\alpha)$ to the coefficients of $f$,
and the six embeddings of $\Q(\beta)$ to $g$.  The resulting list of $9$
$q$-expansions will be called ${\tt f\_conjs}$.

In [65]:
f_conjs = [Q1092q([Q1092(e(c)) for c in f.list()]) for e in emb3]
g_conjs = [Q1092q([Q1092(e(c)) for c in g.list()]) for e in emb8]
f_conjs = f_conjs + g_conjs

Now these $9$ ${\tt f\_conjs}$ are $q$-expansions of the $9$ normalized
newforms, in $\Qzeta{1092}[[q]]$.  Each $q$-expansion starts
$q+eq^2+\dots$ where $e$ is the $T_2$-eigenvalue:

In [66]:
assert all ([fi.list()[:3]==[0,1,e] for fi,e in zip(f_conjs,eig9)])

We also verify that twisting by $\chi$ cycles the newforms round as it
should, at least for the first $20$ $q$-coefficients:

In [67]:
assert all([all([gi==chi(i)*fi
            for i,fi,gi in zip(range(20),
                           list(f_conjs[j]),
                           list(f_conjs[(j+3)%9]))])
            for j in range(9)])

We now construct a basis for the $A_4$-invariant cusp forms by taking
suitable linear combinations of the $9$ newforms.  Then we will find
the algebraic relation between them which gives a model for the
associated modular curve.

Recall that the first $A_4$-invariant cusp form is a linear
combination of $f$ and its twists by the cubic character $\chi$.  The
coefficients in this linear combination have been computed above as
${\tt coeffs\_scaled}$, but they also need to be complex conjugated.
See Corollary 3.16 in my notes for this; it is not easy to see why
this complex conjugation is necessary, but it concerns the duality
between modular symbols and cusp forms which is a duality over $\R$
but not quite over $\C$.

In [68]:
coeffs_scaled_conj = [[conj1092(c) for c in ci] for ci in coeffs_scaled]

Now,  at last, we can form three $q$-expansions giving  a basis for the
$A_4$-invariant cusp forms.

In [69]:
hs = [sum([coeffs_scaled_conj[j][i]*f_conjs[3*i+j]
        for i in range(3)]) for j in range(3)]

These $q$-expansions (in ${\tt hs}$) have coefficients in
$\Qzeta{1092}$, and in fact they lie in a subfield of $\Qzeta{1092}$
of degree $9$, namely the composite $\Q(\zeta_7^+,\zeta_{13}^{++})$.
But there is a basis which has coefficients in the smaller field
$\Qzetaplusplus{13}$ (though not $\Q$) which we find by Galois theory.
Later we will make a second change of basis over $\Q$ which (as
it will turn out) simplifies the quartic equation for the modular
curve, moving its four rational points to $[1:0:0]$, $[0:1:0]$, $[0:0:1]$ and $[1:1:1]$.

In [70]:
MK7=Matrix([[e^j for j in range(3)] for e in eig3])
MM = MK7^(-1)
hh = list(MM * vector(hs))

## Finding the quartic relation giving the equation for $X_{A_4}(13)$

The three new $q$-expansions in ${\tt hh}$ still have $\Qzeta{1092}[[q]]$ as
their parent, though they do in fact lie in $\Qzetaplusplus{13}[[q]]$; we use the
embedding $\Qzeta{13}\hookrightarrow\Qzeta{1092}$ to redefine them as
elements of $\Qzeta{13}[[q]]$.  This gives a new triple of $q$-expansions
called ${\tt A4\_forms}$ which are mathematically the same as ${\tt hh}$ but are
in $\Qzeta{13}[[q]]$.

In [71]:
A4_forms = [Q13q([eQ13Q1092.preimage(c) for c in h]) for h in hh]

We partially echelonise these, to obtain a basis for the $3$-dimensional space they span which is independent of any previous non-determinitic steps (or results which may change with newer versions of the software).  It's important to only use a change of basis matrix over $\Q$ here, as otherwise the quartic we construct will not have rational coefficients.

The second transformation then ensures that the four rational points are the simplest possible (after first constructing the curve with the second transformation, searching for points and then working out the appropriate change of variables).

In [72]:
M = Matrix([[f[i][0] for i in range(1,4)] for f in A4_forms]).inverse()
A4_forms = [sum([Mij*f for Mij, f in zip(r,A4_forms)]) for r in M.rows()]
M1 = M
#MM = MM * M.transpose().inverse()
M = Matrix(QQ,3,3,[9/13, 0, 18/13, 6/13, -6/13, 15/13, -9/26, 27/26, 27/26])
A4_forms = [sum([Mij*f for Mij, f in zip(r,A4_forms)]) for r in M.rows()]
M2 = M
#MM = MM * M.transpose().inverse()

Using these we find a quartic algebraic relation between them, by
finding a linear relation between all the 15 monomials of degree 4.

In [73]:
RXYZ.<X,Y,Z> = QQ[]
mons4 = ((X+Y+Z)^4).monomials()
mons3 = ((X+Y+Z)^3).monomials()
mons2 = ((X+Y+Z)^2).monomials()

We form the 15 monomial combinations of the ${\tt A4\_forms}$ and check that there
is a unique linear relation between them (using the first 30
coefficients in the $q$-expansions):

In [74]:
h4s=[m(A4_forms) for m in mons4]
relmat = Matrix([[list(G)[i] for i in range(30)] for G in h4s])
assert 1 == relmat.nullity()

Now compute that relation.  In fact, although we have been doing
linear algebra over $\Qzeta{13}$, the coefficients are in $\Q$ (and
even in $\Z$ after scaling by a factor $4$):

In [75]:
polA4 = sum([c*m for c,m in zip(relmat.kernel().basis()[0],mons4)])
de = polA4.denominator()
polA4 = de*polA4.change_ring(QQ)

We check that the polynomial relation holds to the
$q$-adic precision used.

In [76]:
assert polA4(A4_forms).valuation() > ps_prec, "cusp forms do not satisfy the quartic relation"

The ternary quartic polynomial we have found is:

In [77]:
polA4

177*X^3*Y - 360*X^2*Y^2 + 189*X*Y^3 + 22*X^3*Z - 294*X^2*Y*Z + 540*X*Y^2*Z - 270*Y^3*Z - 12*X^2*Z^2 - 36*X*Y*Z^2 + 36*Y^2*Z^2 + 56*X*Z^3 - 48*Y*Z^3

Using this homogeneous quartic we construct the curve $X_{A_4}(13)$ as
a plane curve, check that it is smooth and has genus $3$ (as it must,
though I do not have a way of showing *a priori* that it is not
hyperelliptic).   Here we construct the curve over $\Qzeta{13}$ and
not over $\Q$ since we will later construct some points on it,
including the cusps, and these are not rational but defined
over $\Qzeta{13}$.

In [78]:
XA413Q=Curve(polA4)
assert XA413Q.genus()==3, "curve should have genus 3"
assert XA413Q.is_smooth(), "curve should be smooth"
XA413=Curve(polA4.change_ring(Q13))
XA413Q

Projective Plane Curve over Rational Field defined by 177*X^3*Y - 360*X^2*Y^2 + 189*X*Y^3 + 22*X^3*Z - 294*X^2*Y*Z + 540*X*Y^2*Z - 270*Y^3*Z - 12*X^2*Z^2 - 36*X*Y*Z^2 + 36*Y^2*Z^2 + 56*X*Z^3 - 48*Y*Z^3

## Points on $X_{A_4}(13)$: I

Before computing all the $7$ cusps, we construct some "easy'' points,
including $4$ rational points.  These four rational points
have the simplest possible coordinates only because of the change of
basis we used earlier:

In [79]:
PtsQ = [XA413Q(0,0,1), XA413Q(0,1,0),XA413Q(0,0,1),XA413Q(1,1,1)]
# Use the following code to find 4 rational points in the first place
b = 0
while len(PtsQ)<4:
    b += 1
    print("Finding rational points with height bound {}".format(b))
    PtsQ = XA413Q.rational_points(bound=b)
    print("{} points found: {}".format(len(PtsQ),PtsQ))
PtsQ    

[(0 : 0 : 1), (0 : 1 : 0), (0 : 0 : 1), (1 : 1 : 1)]

This is the utility function used to map any 4 points, no 3 of which are collinear, to the standard four:

In [80]:
def map_to_standard_triangle(P1, P2, P3, P4):
    M = Matrix([P1,P2,P3]).transpose().inverse()
    lambdas = M*vector(P4)
    D = diagonal_matrix(M*vector(P4)).inverse()
    M = D * M
    return M

Next, the cusp infinity is easy to compute as its coordinates are
simply the leading coefficients of the three $q$-expansions.  This
gives a point of degree $3$, defined over $\Qzetaplusplus{13}$, and we also
compute its Galois conjugates (which are also cusps):

In [81]:
pt = [f[1] for f in A4_forms]
PtsQ13 = [XA413(pt), XA413([Q13sigma(c) for c in pt]),
                     XA413([Q13sigma(Q13sigma(c)) for c in pt])]
Pts = PtsQ + PtsQ13

So far we have $7$ points, of which $4$
are rational and $3$ are defined over $\Qzeta{13}$
(in fact over $\Qzetaplusplus{13}$):

In [82]:
for P in Pts:
    print(P)

(0 : 0 : 1)
(0 : 1 : 0)
(0 : 0 : 1)
(1 : 1 : 1)
(-4/5*zeta13^11 - 4/5*zeta13^10 - 2/5*zeta13^9 - 2/5*zeta13^7 - 2/5*zeta13^6 - 2/5*zeta13^4 - 4/5*zeta13^3 - 4/5*zeta13^2 - 4/5 : -2/3*zeta13^11 - 2/3*zeta13^10 - 2/3*zeta13^3 - 2/3*zeta13^2 : 1)
(2/5*zeta13^11 + 2/5*zeta13^10 + 4/5*zeta13^9 + 4/5*zeta13^7 + 4/5*zeta13^6 + 4/5*zeta13^4 + 2/5*zeta13^3 + 2/5*zeta13^2 : 2/3*zeta13^11 + 2/3*zeta13^10 + 2/3*zeta13^9 + 2/3*zeta13^7 + 2/3*zeta13^6 + 2/3*zeta13^4 + 2/3*zeta13^3 + 2/3*zeta13^2 + 2/3 : 1)
(2/5*zeta13^11 + 2/5*zeta13^10 - 2/5*zeta13^9 - 2/5*zeta13^7 - 2/5*zeta13^6 - 2/5*zeta13^4 + 2/5*zeta13^3 + 2/5*zeta13^2 - 2/5 : -2/3*zeta13^9 - 2/3*zeta13^7 - 2/3*zeta13^6 - 2/3*zeta13^4 : 1)


## Points on $X_{A_4}(13)$: II cusps

We have an explicit model for the modular curve $X_{A_4}(13)$, defined
over $\Q$, and a parametrization of it (the "canonical
embedding'' into $\P^2$) using the three cusp forms of weight $2$,
${\tt A4\_forms}=(f_1,f_2,f_3)$, which can be evaluated at any point $\tau$
in the upper half-plane to give a point $\alpha(\tau) =
[f_1(\tau):f_2(\tau):f_3(\tau)] \in X_{A_4}(13)(\C)$.  The image of
the upper half-plane under this map $\alpha$ is an open subset of
$X_{A_4}(13)$, omitting the cusps.  The image of the cusp $\infty$ is
easy to determine, since $q$ is a local parameter there and the three
$q$-expansions $f_i$ are expansions of the coordinate functions
$X,Y,Z$ of our model in term of this parameter.  Bearing in mind that
the $f_i$ are cusp forms and so start $q+a_2^{(i)}q^2\dots$ we see that
$\alpha(\infty)=[a_2^{(1)}:a_2^{(2)}:a_2^{(3)}]$ as evaluated above.

What about the other cusps?  These have the form $c=g(\infty)$ for
$g\in\PSL(2,\Z)$, so
\begin{align*}
  \alpha(c) &= [f_1(g(\infty)):f_2(g(\infty)):f_3(g(\infty))]\\
            &= [(f_1|g)(\infty)):(f_2|g)(\infty)):(f_3|g)(\infty))]\\
            &= [(f_1|g)[1]:(f_2|g)[1]:(f_3|g)[1])]
\end{align*}
where the notation $f[1]$ means the coefficient of $q^1$ in a
$q$-expansion.

The cusp forms $f_i|g$ which appear in this formula are in the
$50$-dimensional space of cusp forms of weight $2$ for $G$, and even
in the $36$-dimensional subspace dual to the modular symbol space ${\tt
  Wplus}$, but do not lie in the $9$-dimensional subspace (spanned by
${\tt f\_conjs})$ we used to find the equation.  This is why we
constructed ${\tt Wplus}$.  If we can express each $f_i|g$ explicitly as a linear
combination of newforms then, since newforms are by definition
normalized, we can extract the coefficient of $q^1$ in their expansion
simply by adding the coefficients in the linear combination, without
needing to know all the newforms' $q$-expansions at all.

We now define a function to carry this plan out.  By reverting from
the basis $(f_1,f_2,f_3)$ to the first basis we found for the
$A_4$-invariants, we are able to replace one $36$-dimensional
computation by three $12$-dimensional ones, which are conjugate over
$\Qzetaplus{7}$ so in fact do a single $12$-dimensional computation followed
by Galois conjugation.  As before, we do most of the work in the
modular symbol space.  Suppose that ${\tt gam}$ is the $12\times12$
matrix giving the action of an element $g\in\PSL(2,\Z)$ on the
$12$-dimensional modular symbol space ${\tt W1}$, viewed as a subspace
of $\Qzeta{84}^{100}$.  For example, the generators $S$ and $T$
of $\PSL(2,\Z)$ act via the matrices ${\tt SW1}$ and ${\tt TW1}$.  More
generally, we construct such ${\tt gam}$ by writing $g$ as a word in
$S,T$ and setting ${\tt gam}$ to be the same word in ${\tt SW1}$, ${\tt
  TW1}$.

Given the $12\times12$ matrix ${\tt gam}$ we proceed as follows:

   1. Multiply ${\tt gam}$ by the $A_4$-invariant vector ${\tt
  A4\_inv\_W1}$ in ${\tt W1}$, and then by the basis matrix of ${\tt W1}$ to
  get a vector in $\Qzeta{84}^{100}$, then express this as a linear
  combination of the eigenbasis for ${\tt W1}$.
   2.  Apply the order $3$ Galois automorphism to the resulting vector
  (denoted ${\tt v1}$ in the code below, and of length $12$) get two more
  vectors (denoted ${\tt v2}$ and ${\tt v3}$); these three vectors are a
  basis for the image of the $A_4$-invariant vectors under ${\tt gam}$.
   3. Change basis (using matrix ${\tt MM}$) and scale by Gauss sums and
  complex conjugation to get the coordinates of the images of
  $f_1,f_2,f_3$ under $g$ with respect to the newform basis, and add
  up these coordinates to obtain the $q^1$-coefficient.  (In the code
  below this is done using an inner product.)
   4. This gives a vector of three homogeneous coordinates, in
  $\Qzeta{84}$, of the desired point; finally divide by the last
  nonzero coordinate and pull back to $\Qzeta{13}$ to get a point
  in $X_{A_4}(13)(\Qzeta{13})$.

This is achieved by the following Sage function:

In [83]:
def get_cusp(gam): # returns coords of cusp gam(infinity)
     v1 = Mlongevec12.solve_left(A4_inv_W1*gam*W1.basis_matrix())
     v2 = [aut7(c) for c in v1]
     v3 = [aut7(c) for c in v2]
     p2 = list(M2*M1*MM*Matrix([v1,v2,v3])*gepsvecm)
     p2 = [conj1092(c) for c in p2] # complex conjugation
     p2 = [c/p2[2] for c in p2]     # normalize so last coordinate is 1
     assert polA4(p2)==0
     p2 = [eQ13Q1092.preimage(c) for c in p2] # pull back to Q13
     assert polA4(p2)==0
     return p2

For example and as a check, taking ${\tt gam}$ to be the identity matrix, the cusp $\infty$ itself maps to
$\alpha(\infty) = $

In [84]:
XA413(get_cusp(TW1.parent()(1)))

(-4/5*zeta13^11 - 4/5*zeta13^10 - 2/5*zeta13^9 - 2/5*zeta13^7 - 2/5*zeta13^6 - 2/5*zeta13^4 - 4/5*zeta13^3 - 4/5*zeta13^2 - 4/5 : -2/3*zeta13^11 - 2/3*zeta13^10 - 2/3*zeta13^3 - 2/3*zeta13^2 : 1)

All but one of the $7$ cusps for $G$ are represented by integers $j$
in $\P^1(\Q)$: one can check that modulo $G$ we have
$$
   0\sim\infty, \qquad
   1\sim5\sim8\sim12, \qquad
   2\sim7, \qquad
   3\sim4, \qquad
   6\sim11, \qquad
   9\sim10;
$$
we represent these as $g(\infty)$ where $g=T^jS$.
The last cusp is $7/6 = g(\infty)$ for $g=TST^{-6}S$.  (The
computation of cusp equivalences was done separately.) 

Here is a function for the integral cusps:

In [85]:
def get_integral_cusp(j):
     return get_cusp(TW1^j*SW1)

which we apply to get $6$ of the $7$ cusps:

In [86]:
XA413_cusps = [XA413(get_integral_cusp(j)) for j in [0,1,2,9,6,3]]

For the last cusp we have

In [87]:
gam = TW1*SW1*TW1^(-6)*SW1

since

In [88]:
TZ*SZ*TZ^(-6)*SZ

[-7 -1]
[-6 -1]

maps $\infty\mapsto 7/6$.

In [89]:
P = XA413(get_cusp(gam))
assert not P in XA413_cusps, "this cusp should be a 7th one"
XA413_cusps.insert(2,P)

So we now have all $7$ cusps  as points in
$X_{A_4}(13)(\Qzeta{13})$:

In [90]:
for P in XA413_cusps: print(P)

(-4/5*zeta13^11 - 4/5*zeta13^10 - 2/5*zeta13^9 - 2/5*zeta13^7 - 2/5*zeta13^6 - 2/5*zeta13^4 - 4/5*zeta13^3 - 4/5*zeta13^2 - 4/5 : -2/3*zeta13^11 - 2/3*zeta13^10 - 2/3*zeta13^3 - 2/3*zeta13^2 : 1)
(2/5*zeta13^11 + 2/5*zeta13^10 + 4/5*zeta13^9 + 4/5*zeta13^7 + 4/5*zeta13^6 + 4/5*zeta13^4 + 2/5*zeta13^3 + 2/5*zeta13^2 : 2/3*zeta13^11 + 2/3*zeta13^10 + 2/3*zeta13^9 + 2/3*zeta13^7 + 2/3*zeta13^6 + 2/3*zeta13^4 + 2/3*zeta13^3 + 2/3*zeta13^2 + 2/3 : 1)
(2/5*zeta13^11 + 2/5*zeta13^10 - 2/5*zeta13^9 - 2/5*zeta13^7 - 2/5*zeta13^6 - 2/5*zeta13^4 + 2/5*zeta13^3 + 2/5*zeta13^2 - 2/5 : -2/3*zeta13^9 - 2/3*zeta13^7 - 2/3*zeta13^6 - 2/3*zeta13^4 : 1)
(-18/61*zeta13^11 - 14/61*zeta13^9 - 18/61*zeta13^8 - 18/61*zeta13^7 - 2/61*zeta13^6 - 2/61*zeta13^5 - 14/61*zeta13^3 - 2/61*zeta13^2 - 14/61*zeta13 + 62/61 : -4/9*zeta13^11 - 4/9*zeta13^9 - 4/9*zeta13^8 - 4/9*zeta13^7 - 2/9*zeta13^6 - 2/9*zeta13^5 - 4/9*zeta13^3 - 2/9*zeta13^2 - 4/9*zeta13 + 2/3 : 1)
(4/61*zeta13^11 + 16/61*zeta13^9 + 4/61*zeta13^8 + 4/6

Their degrees are

In [91]:
[max([co.minpoly().degree() for co in list(c)]) for c in XA413_cusps]

[3, 3, 3, 4, 4, 4, 4]

## The $j$-map $X_{A_4}(13)\longrightarrow X(1)$

We now find an expression for the rational map $j$ of degree $91$ from
$X_{A_4}(13)$ to the $j$-line~$X(1)$.

The poles of $j$ are at the $7$ cusps which we have computed exactly,
each with multiplicity $13$.  The zeroes of $j$ are harder to pin down.
One approach, as used by Burcu Baran for $\Xsplit(13)$ and
$\Xnonsplit(13)$, is to use the parametrisation of $X_{A_4}(13)$ by
modular functions on the upper half-plane.  Write $X,Y,Z$ for the
three cusp forms denoted ${\tt A4\_forms}$ above.  Then the parametrizing
map
$$
   \varphi: {\mathcal H} \to X_{A_4}(13)(\C)
$$
is given by $\tau\mapsto [X(q):Y(q):Z(q)]$ with $q=\exp(2\pi i\tau)$.
We can extend this map to the cusps as indicated in the previous
section.   Now the zeroes of $j$ are the points $\varphi(\tau)$ such
that $j(\tau)=0$: there are $91$ of these, counting multiplicities (in
fact $29$ have multiplicity $3$ and $4$ have multiplicity $1$), coming
from the elliptic points of order $3$ on the upper half-plane
$\mathcal{H}$. 

Baran's approach is to compute $\varphi(\tau)$ numerically from the
$q$-expansions for $91$ (or in fact $29+4=33$) elliptic points $\tau$
and hence recognise these as algebraic points.  If successful, then
one has the complete divisor of $j$ as a rational function on the
curve and can use Riemann-Roch to find $j$.

Instead we proceed as follows.  First we find a polynomial in $X,Y,Z$
passing through the $7$ cusps.  There are no
quadratics through these points:

In [92]:
mons2 = ((X+Y+Z)^2).monomials()
matrix([[m(list(c)) for c in XA413_cusps] for m in mons2]).nullity()

0

but there are cubics:

In [93]:
mons3 = ((X+Y+Z)^3).monomials()
ker = matrix([[m(list(c)) for c in XA413_cusps] for m in mons3]).left_kernel()
ker.dimension()

3

We find an LLL-reduced basis for the integral cubics in this
$3$-dimensional space:

In [94]:
M = ker.basis_matrix()
M = (M*M.denominator()).change_ring(ZZ)
Sm,U,V = M.smith_form()
M = (Sm.submatrix(ncols=3)^(-1)*U*M).change_ring(ZZ)
M = (M*M.transpose()).LLL_gram().transpose() * M
cubs = [sum([c*m for c,m in zip(r,mons3)]) for r in M.rows()]

This gives the following cubics:

In [95]:
for cub in cubs:
    print(cubs)

[51*X^2*Y - 36*X*Y^2 - 4*X^2*Z - 60*X*Y*Z + 36*Y^2*Z - 16*X*Z^2 + 12*Y*Z^2 + 24*Z^3, 53*X^2*Y - 51*X*Y^2 + 27*Y^3 - 52*X^2*Z + 88*X*Y*Z - 78*Y^2*Z - 60*X*Z^2 + 40*Y*Z^2 + 40*Z^3, -305*X^3 + 398*X^2*Y - 501*X*Y^2 + 189*Y^3 + 32*X^2*Z + 238*X*Y*Z + 84*Y^2*Z + 584*X*Z^2 - 464*Y*Z^2 - 344*Z^3]
[51*X^2*Y - 36*X*Y^2 - 4*X^2*Z - 60*X*Y*Z + 36*Y^2*Z - 16*X*Z^2 + 12*Y*Z^2 + 24*Z^3, 53*X^2*Y - 51*X*Y^2 + 27*Y^3 - 52*X^2*Z + 88*X*Y*Z - 78*Y^2*Z - 60*X*Z^2 + 40*Y*Z^2 + 40*Z^3, -305*X^3 + 398*X^2*Y - 501*X*Y^2 + 189*Y^3 + 32*X^2*Z + 238*X*Y*Z + 84*Y^2*Z + 584*X*Z^2 - 464*Y*Z^2 - 344*Z^3]
[51*X^2*Y - 36*X*Y^2 - 4*X^2*Z - 60*X*Y*Z + 36*Y^2*Z - 16*X*Z^2 + 12*Y*Z^2 + 24*Z^3, 53*X^2*Y - 51*X*Y^2 + 27*Y^3 - 52*X^2*Z + 88*X*Y*Z - 78*Y^2*Z - 60*X*Z^2 + 40*Y*Z^2 + 40*Z^3, -305*X^3 + 398*X^2*Y - 501*X*Y^2 + 189*Y^3 + 32*X^2*Z + 238*X*Y*Z + 84*Y^2*Z + 584*X*Z^2 - 464*Y*Z^2 - 344*Z^3]


We do not use the first of these as it passes through some
of the rational points we know on the curve, which will cause problems
later when we try to evaluate $j$ on these points:

In [96]:
for cub in cubs:
    vals = [cub(list(p)) for p in PtsQ]
    print("cubic {} has values {} at the rational points".format(cub,vals))
    if not any([v==0 for v in vals]):
        print("-- values do not include 0, so use this one")
        break


cubic 51*X^2*Y - 36*X*Y^2 - 4*X^2*Z - 60*X*Y*Z + 36*Y^2*Z - 16*X*Z^2 + 12*Y*Z^2 + 24*Z^3 has values [24, 0, 24, 7] at the rational points
cubic 53*X^2*Y - 51*X*Y^2 + 27*Y^3 - 52*X^2*Z + 88*X*Y*Z - 78*Y^2*Z - 60*X*Z^2 + 40*Y*Z^2 + 40*Z^3 has values [40, 27, 40, 7] at the rational points
-- values do not include 0, so use this one


Now we convert the $q$-expansions of the three $A_4$-invariant cusp
forms to lie in $\Qzetaplusplus{13})[[q]]$; at the same time we divide
each by $q$ (otherwise all the power series exponents later will be
unnecessarily shifted by $39$).

In [97]:
ffq = [ Q13ppq([eQ13ppQ13.preimage(c) for c in list(f)[1:]]).add_bigoh(ps_prec)
        for f in A4_forms]
xq,yq,zq = ffq 

The denominator of $j$ may be taken to be the $13$th power of the
cubic through the cusps:

In [98]:
denj13 = cub(ffq)^13

For $j$ itself we must evaluate the standard $q$-expansion for the
$j$-function at $q^{13}$:

In [99]:
jq = j_invariant_qexp(20, Q13pp)
jq13 = jq(Q13ppq.gen()^13)

Now the numerator of the rational function is a homogeneous polynomial
of degree $39$ whose coefficients we must determine so that
when evaluated at the $q$-expansions of $X,Y,Z$ equals $j(q^{13})$
times the denominator:

In [100]:
numjq39 = jq13*denj13

We set up a system of linear equations to solve for the unknown
coefficients.  We do not need to use all monomials of degree $39$
since the curve equation can be used to reduce the numerator polynomial.

In [101]:
mons39 = ((X+Y+Z)^39).monomials()
len(mons39)
mons39r = [m for m in mons39 if m.degrees()[0]<3 or m.degrees()[1]==0]
len(mons39r)

154

So we need at least $154$ coefficients of the
$q$-expansions.  For efficiency, instead of simply evaluating all the
monomials at $X,Y,Z$ we first find all necessary powers of each
variable and then combine these.

In [102]:
xpowers = [xq.parent()(1)]
ypowers = [yq.parent()(1)]
zpowers = [zq.parent()(1)]
while len(xpowers)<40:
    xpowers += [xq*xpowers[-1]]
    ypowers += [yq*ypowers[-1]]
    zpowers += [zq*zpowers[-1]]

mons39q = [xpowers[m.degrees()[0]]*ypowers[m.degrees()[1]]*zpowers[m.degrees()[2]] for m in mons39r]

We extract the array of the relevant coefficients; it is enough to
look at the coefficients of $q^n$ for $n<160$.  Significantly,
since we know that the solution will be $1$-dimensional and with
coordinates in $\Q$, we can save a lot of time by reducing to linear
algebra over $\Q$:

In [103]:
arr = [ [mq[i] for i in range(160)] for mq in mons39q] + \
        [[-numjq39[i] for i in range(160)]]
relmatQ =  Matrix([  flatten([list(aij) for aij in ai])  for ai in arr ])
ker = relmatQ.left_kernel()

If all is well the dimension of the solution space should be $1$:

In [104]:
ker.dimension()

1

We extract the coefficients from the coordinates of the solution
vector:

In [105]:
b = ker.basis()[0]
b *= b.denominator()
d = b[-1]

and form the numerator and denominator of the rational function we
have been seeking:

In [106]:
jnum = sum([bi*m for bi,m in zip(list(b)[:-1],mons39r)])
jden = d*cub^13

The images of the rational points on $X_{A_4}(13)$ give rational
$j$-invariants of elliptic curves whose projective mod-$13$ Galois
representation is contained in $S_4$ (and in $A_4$ after base change
to $\Q(\sqrt{13})$):

In [107]:
[(jnum(list(p))/jden(list(p))) for p in PtsQ]

[-160855552000/1594323, 11225615440/1594323, -160855552000/1594323, 0]

If we knew any points defined over other fields such as $\Q(\sqrt{13})$ we could evaluate the j-map on them.  But the only extra points we know about (so far) are all cusps:

In [108]:
[jden(list(p)) for p in PtsQ13]

[0, 0, 0]

## The Split Cartan case:  $X_{s}(13)$

This is the simplest of the three cases, since the space of cusp forms
invariant under the split Cartan normaliser is spanned by $3$
conjugate newforms with trivial character.  We choose the split Cartan
subgroup to be the subgroup of diagonal matrices: then a cusp form is
invariant under the split Cartan if and only if it has trivial
character, and is also invariant under the normalizer of the split
Cartan if it has eigenvalue $+1$ for the Atkin-Lehner involution.  The
space of such forms is $3$-dimensional and is the space spanned by the
first three of the $q$-expansions denoted ${\tt f\_conjs}$ above.


### An equation for the curve

We make a change of basis here so that the equation we obtain agrees
with that obtained by Burcu Baran.

In [109]:
M1 = Matrix(QQ,3,3,[-1, 1, -3, -1, 0, -1, 0, 0, 1])
SC_forms = M1 * MK7^(-1)  * vector(f_conjs[:3])
SC_forms = [QQq(list(gi)).add_bigoh(ps_prec) for gi in SC_forms]

We find a quartic relation between these just as before;  the
computation is simpler since these $q$-expansions have rational
coefficients.

In [110]:
g4s=[m([f.add_bigoh(40) for f in SC_forms]) for m in mons4]
relmat = Matrix([[list(G)[i] for i in range(30)] for G in g4s])

We find the polynomial (changing its sign so that ${\tt
  polSC}$ is exactly the same as Burcu's defining polynomial):

In [111]:
relmat.nullity()

1

In [112]:
polSC = -sum([c*m for c,m in zip(relmat.kernel().basis()[0],mons4)])
polSC(SC_forms).valuation() > ps_prec
polSC

-X^3*Y + 2*X^2*Y^2 - X*Y^3 - X^3*Z + X^2*Y*Z + X*Y^2*Z - 2*X*Y*Z^2 + 2*Y^2*Z^2 + X*Z^3 - 3*Y*Z^3

Now we construct the curve (over $\Qzeta{13})$ to accommodate the cusps)
and check that it is smooth and of genus $3$:

In [113]:
XSC13 = Curve(polSC.change_ring(Q13))
XSC13.is_smooth()
Curve(polSC).genus()

3

### Cusps and rational points

The cusps are computed as before.

In [114]:
SC_inv_W1 = vector(W1.coordinates(longevec12[0]))

In [115]:
def get_SC_cusp(gam): # returns coords of cusp gam(infinity)
       v1 = Mlongevec12.solve_left(SC_inv_W1*gam*W1.basis_matrix())
       v2 = [aut7(c) for c in v1]
       v3 = [aut7(c) for c in v2]
       p2 = list(M1*MK7^(-1)*Matrix([v1,v2,v3])*gepsvecm)
       p2 = [conj1092(c) for c in p2] # complex conjugation
       if p2[2]!=0:                   # normalize so last coordinate is 1
           p2 = [c/p2[2] for c in p2]
       else:
           if p2[1]!=0:
               p2 = [c/p2[1] for c in p2]
           else:
               p2 = [c/p2[0] for c in p2]
       assert polSC(p2)==0
       p2 = [eQ13Q1092.preimage(c) for c in p2] # pull back to Q13
       assert polSC(p2)==0
       return p2

def get_integral_SC_cusp(j):
       return get_SC_cusp(TW1^j*SW1)

In [116]:
XSC13_cusps = [XSC13(get_integral_SC_cusp(j)) for j in range(7)]

This really does find the all $7$ cusps, since
the integers $\{0,1,2,3,4,5,6\}$ represent the $7$ cusps, each of
width $13$.

In [117]:
for P in XSC13_cusps: print(P)

(1 : 1 : 0)
(-zeta13^11 - zeta13^10 - zeta13^7 - zeta13^6 - zeta13^3 - zeta13^2 : -zeta13^10 - zeta13^9 - zeta13^8 - zeta13^7 - zeta13^6 - zeta13^5 - zeta13^4 - zeta13^3 : 1)
(zeta13^11 + zeta13^9 + zeta13^7 + zeta13^6 + zeta13^4 + zeta13^2 + 1 : -zeta13^11 - zeta13^10 - zeta13^9 - zeta13^8 - zeta13^5 - zeta13^4 - zeta13^3 - zeta13^2 : 1)
(zeta13^10 + zeta13^9 + zeta13^7 + zeta13^6 + zeta13^4 + zeta13^3 + 1 : zeta13^9 + zeta13^8 + zeta13^5 + zeta13^4 + 1 : 1)
(-zeta13^9 - zeta13^8 - zeta13^7 - zeta13^6 - zeta13^5 - zeta13^4 : zeta13^10 + zeta13^7 + zeta13^6 + zeta13^3 + 1 : 1)
(-zeta13^11 - zeta13^10 - zeta13^9 - zeta13^4 - zeta13^3 - zeta13^2 : zeta13^10 + zeta13^8 + zeta13^5 + zeta13^3 + 1 : 1)
(zeta13^11 + zeta13^10 + zeta13^8 + zeta13^5 + zeta13^3 + zeta13^2 + 1 : zeta13^11 + zeta13^9 + zeta13^4 + zeta13^2 + 1 : 1)


A quick search find a few rational points:

In [118]:
XSC13Q = [XSC13(list(P)) for P in Curve(polSC).rational_points(bound=5)]
XSC13Q

[(-1 : 0 : 1),
 (0 : 0 : 1),
 (0 : 1 : 0),
 (0 : 3/2 : 1),
 (1 : 0 : 0),
 (1 : 0 : 1),
 (1 : 1 : 0)]

Only one cusp is rational, the others are conjugate over $\Q(\zeta_{13}^{+})$:

In [119]:
print([P for P in XSC13_cusps if P in XSC13Q])
print(XSC13_cusps[1])
[max([co.minpoly().degree() for co in list(c)]) for c in XSC13_cusps]

[(1 : 1 : 0)]
(-zeta13^11 - zeta13^10 - zeta13^7 - zeta13^6 - zeta13^3 - zeta13^2 : -zeta13^10 - zeta13^9 - zeta13^8 - zeta13^7 - zeta13^6 - zeta13^5 - zeta13^4 - zeta13^3 : 1)


[1, 6, 6, 6, 6, 6, 6]

### Map to the $j$-line

Now we find the rational function giving the map to the $j$-line.  For
the denominator, we check that there are no quadratics through the
seven cusps:

In [120]:
matrix([[m(list(c)) for c in XSC13_cusps] for m in mons2]).nullity() == 0

True

So we find the cubics, which again form a $3$-dimensional space:

In [121]:
ker = matrix([[m(list(c)) for c in XSC13_cusps] for m in mons3]).left_kernel()
ker.dimension()

3

We clear denominators, saturate and LLL-reduce:

In [122]:
M = ker.basis_matrix()
M = (M*M.denominator()).change_ring(ZZ)
Sm,U,V = M.smith_form()
M = (Sm.submatrix(ncols=3)^(-1)*U*M).change_ring(ZZ)
M = (M*M.transpose()).LLL_gram().transpose() * M
cubs = [sum([c*m for c,m in zip(r,mons3)]) for r in M.rows()]
cubs

[-X^2*Y + X*Y^2 + X*Y*Z - Y^2*Z - 2*X*Z^2 + 2*Y*Z^2 - Z^3,
 -X^3 - X^2*Y + 2*X*Y^2 + 3*X^2*Z - 2*X*Y*Z - Y^2*Z + Y*Z^2 - Z^3,
 2*X^2*Y - 3*X*Y^2 + Y^3 + 2*X^2*Z + 2*X*Y*Z - 3*Y^2*Z + X*Z^2 + Y*Z^2]

There is a rational cusp, and we will want to choose a cubic for the
denominator which does not pass through any of the other rational
points:

In [123]:
XSC13Q_noncusp = [P for P in XSC13Q if not P in XSC13_cusps]

In [124]:
for cub in cubs:
    print(cub, [cub(list(p)) for p in XSC13Q_noncusp])

cub=cubs[1]+cubs[2]
assert all([cub(list(p))!=0 for p in XSC13Q_noncusp])
assert all([cub(list(p))==0 for p in XSC13_cusps])
cub

-X^2*Y + X*Y^2 + X*Y*Z - Y^2*Z - 2*X*Z^2 + 2*Y*Z^2 - Z^3 [1, -1, 0, -1/4, 0, -3]
-X^3 - X^2*Y + 2*X*Y^2 + 3*X^2*Z - 2*X*Y*Z - Y^2*Z + Y*Z^2 - Z^3 [3, -1, 0, -7/4, -1, 1]
2*X^2*Y - 3*X*Y^2 + Y^3 + 2*X^2*Z + 2*X*Y*Z - 3*Y^2*Z + X*Z^2 + Y*Z^2 [1, 0, 1, -15/8, 0, 3]


-X^3 + X^2*Y - X*Y^2 + Y^3 + 5*X^2*Z - 4*Y^2*Z + X*Z^2 + 2*Y*Z^2 - Z^3

To solve for the numerator we first shift the $q$-expansions (dividing
by $q$), evaluate our cubic at the resulting point and multiply
by $j(q^{13})$:

In [125]:
ffq = [f.shift(-1) for f in SC_forms]
denj = cub(ffq)
denj13 = denj^13
jq = j_invariant_qexp(20)
q = jq.parent().gen()
jq13 = jq(q^13)
numjq39 = jq13*denj13

We compute an independent set of monomials of degree $39$ in the
$q$-expansions:

In [126]:
xq,yq,zq = ffq
xpowers = [xq.parent()(1)]
ypowers = [yq.parent()(1)]
zpowers = [zq.parent()(1)]
while len(xpowers)<40:
       xpowers += [xq*xpowers[-1]]
       ypowers += [yq*ypowers[-1]]
       zpowers += [zq*zpowers[-1]]

mons39r = [m for m in mons39 if m.degrees()[0]<2 or m.degrees()[1]<2]
mons39q = [xpowers[m.degrees()[0]]*ypowers[m.degrees()[1]]*zpowers[m.degrees()[2]] for m in mons39r]

Extract the coefficients and form the matrix whose kernel gives the
coefficients of the solution:

In [127]:
relmatQ =  Matrix([[mq[i] for i in range(190)] for mq in mons39q])
sol = relmatQ.solve_left(vector([numjq39[i] for i in range(190)]))

Using these coefficients we form the numerator and denominator as
polynomials in $\Q[X,Y,Z]$:

In [128]:
c0 = sol.denominator()
c = list(c0*sol)
jnum = sum([ci*m for ci,m in zip(c,mons39r)])
jden = c0*cub^13

We do not display the rational function since it is huge, but we do
evaluate it at some points on the curve.

### $j$-invariants of rational points

Now we evaluate the map at the rational points (excluding the cusp):

In [129]:
jlist = [QQ(jnum(list(p))/jden(list(p))) for p in XSC13Q_noncusp]
jlist

[287496, -12288000, 54000, 0, -884736000, 1728]

It appears that these $j$-invariants are all CM, which we check:

In [130]:
all([j in cm_j_invariants(QQ) for j in jlist])

True

Finally we check that the corresponding orders agree with Burcu
Baran's Table 1.1:

In [131]:
CMQ = cm_j_invariants_and_orders(QQ)
[(P,d*f^2) for P,jj in zip(XSC13Q_noncusp,jlist) for (d,f,j) in CMQ if j==jj]

[((-1 : 0 : 1), -16),
 ((0 : 0 : 1), -27),
 ((0 : 1 : 0), -12),
 ((0 : 3/2 : 1), -3),
 ((1 : 0 : 0), -43),
 ((1 : 0 : 1), -4)]

## The Nonsplit Cartan case:  $X_{ns}(13)$

The space of cusp forms invariant under the split Cartan normaliser is
a $3$-dimensional subspace of the $36$-dimensional space of cusp forms
spanned by newforms: ${\tt f\_conjs}$ and all $12$ twists of each.

We choose the nonsplit Cartan subgroup as follows.

In [132]:
B=A
A=T*S*T^(-1)*S*T*S*T^(-5)*S*T^(-1)

Recall that these are $100\times100$ matrices giving the action of
$\PSL(2,13)$ on the full modular symbol space.  Their restrictions to
the irreducible $12$-dimensional space defined over the cubic field
$\Q(\zeta_{13}^{++})$ are

In [133]:
A2W1=TW1*SW1*TW1^(-1)*SW1*TW1*SW1*TW1^(-5)*SW1*TW1^(-1)
B2W1=AW1

which we check satisfy the defining relations for the nonsplit Cartan
normaliser which is isomorphic to the dihedral group of order $14$:

In [134]:
(A2W1^7)==1 and (B2W1^2)==1 and (B2W1*A2W1)^2==1

True

We construct the projector onto the invariant subspace:

In [135]:
sigmaW1 = (1+A2W1+A2W1^2+A2W1^3+A2W1^4+A2W1^5+A2W1^6)*(1+B2W1)
sigmaW1.rank()==1 and sigmaW1*(sigmaW1-14)==0

True

Check that the invariant subspace is $1$-dimensional and extract a
basis vector:

In [136]:
sigmaW1.row_space().dimension()
NS_inv_W1 = sigmaW1.row_space().basis()[0]
[NS_inv_W1*g == NS_inv_W1 for g in [A2W1,B2W1]]

[True, True]

The coordinates of ${\tt NS\_inv\_W1}$ are with respect to the echelon
basis of ${\tt W1}$, but we need the coordinates with respect to the
eigenvector basis:

In [137]:
v = Mlongevec12.solve_left(NS_inv_W1*W1.basis_matrix())

In fact only the even twists occur here:

In [138]:
all([(v[i]==0)==(i%2==1) for i in range(12)])

True

### $q$-expansions

We find the $q$-expansions of all $12$ twists of the first newform:

In [139]:
f12_conjs = [Q1092q([(eps^j)(n)*(an)
           for n,an in enumerate(list(f_conjs[0]))]).add_bigoh(ps_prec)
           for j in range(12)]

Using the coefficient vector ${\tt v}$ we form the $q$-expansion of a
cusp form invariant under the nonsplit Cartan normalizer.  This has
been constructed with 
coefficients in $\Q(\zeta_{1092})$ but in fact has coefficients in
$\Q(\zeta_{91})$, so we pull it back to $\Q(\zeta_{91})[[q]]$, and
then form the three Galois conjugates:

In [140]:
h0 = sum([conj1092(v[i]*gepsvec[i])*f12_conjs[i] for i in range(12)])
h0 = Q91q([eQ91Q1092.preimage(c) for c in list(h0)]).add_bigoh(ps_prec)
h1 = Q91q([Q91aut(c) for c in list(h0)]).add_bigoh(ps_prec)
h2 = Q91q([Q91aut(c) for c in list(h1)]).add_bigoh(ps_prec)
hh=[h0,h1,h2]

By taking suitable linear combinations we arrive at a basis
in $\Q(\zeta_{13})[[q]]$: 

In [141]:
a0=eQ91Q1092.preimage(Q1092(1/zeta7p))
a1=Q91aut(a0)
a2=Q91aut(a1)
MK7 = Matrix([[a^j for a in [a0,a1,a2]] for j in range(3)])
hh = MK7 * vector(hh)
hh = [Q13q([eQ13Q91.preimage(c)for c in h]).add_bigoh(ps_prec) for h in hh]

Finally we make a change of basis so that the quartic equation we find
later is exactly the same as that obtained by Burcu Baran:

In [142]:
MBB = Matrix([[2,1,1],[-6,4,-3],[11,-5,2]])
NS_forms = list(MBB^(-1)*vector(hh))

### Equation of the curve

We find a quartic relation between the three $q$-expansions in the
same way as before.

In [143]:
h4s=[m([h.add_bigoh(40) for h in NS_forms]) for m in mons4]
relmat = Matrix([[list(G)[i] for i in range(30)] for G in h4s])
assert 1==relmat.nullity()
polNS = -sum([c*m for c,m in zip(relmat.kernel().basis()[0], mons4)])
polNS = polNS.change_ring(QQ)

The polynomial we find is

In [144]:
polNS

-X^4 + 40879/7410*X^3*Y - 1441/130*X^2*Y^2 + 65203/7410*X*Y^3 - 12721/7410*Y^4 - 30409/7410*X^3*Z + 43417/2470*X^2*Y*Z - 187333/7410*X*Y^2*Z + 2144/195*Y^3*Z - 45793/7410*X^2*Z^2 + 76948/3705*X*Y*Z^2 - 12359/741*Y^2*Z^2 - 37067/7410*X*Z^3 + 5209/570*Y*Z^3 - 6112/3705*Z^4

which would be the same as the equation for the Split Cartan if we made the appropriate change of basis.

- Unfortunately these computations are not quite deterministic as different versions of Sage may compute different, though mathematically consistent, bases for various spaces.  Our original computation produced the exact same equation for the split and non-split curves, but now this is no longer the case and we have not written code to find the isomorphism.

In [145]:
polNS == polSC

False

Now we construct the curve (over $\Q(\zeta_{13})$ to accommodate the
cusps again) and check that it is smooth and of genus~$3$.  Although
it is the same curve as before we'll construct it anyway.

In [146]:
XNS13 = Curve(polNS.change_ring(Q13))
assert XNS13.is_smooth()
Curve(polNS).genus()

3

### Cusps and rational points

The rational points:

In [147]:
XNS13Q = [XNS13(list(p)) for p in Curve(polNS).rational_points(bound=20)]
XNS13Q

[(-17 : -9 : 1),
 (-10/9 : -1/3 : 1),
 (-8/15 : 8/15 : 1),
 (9/14 : 17/14 : 1),
 (7/8 : 3/4 : 1),
 (2 : 2 : 1)]

Computing the cusps is easier than before, since they are all
Galois conjugates of the cusp at $\infty$ which is easy to obtain from
the $q$-expansions.

In [148]:
XNS13_cusps = [XNS13([(Q13aut^j)(h[1]) for h in NS_forms]) for j in range(6)]

The first cusp is

In [149]:
XNS13_cusps[0]

(48088/36451*zeta13^11 + 162223/182255*zeta13^10 + 15787/36451*zeta13^9 + 138672/182255*zeta13^8 + 243999/182255*zeta13^7 + 243999/182255*zeta13^6 + 138672/182255*zeta13^5 + 15787/36451*zeta13^4 + 162223/182255*zeta13^3 + 48088/36451*zeta13^2 + 11311/182255 : 112181/182255*zeta13^11 + 21089/36451*zeta13^10 + 92632/182255*zeta13^9 + 170761/182255*zeta13^8 + 214564/182255*zeta13^7 + 214564/182255*zeta13^6 + 170761/182255*zeta13^5 + 92632/182255*zeta13^4 + 21089/36451*zeta13^3 + 112181/182255*zeta13^2 + 131088/182255 : 1)

### Map to the $j$-line

As before there are no quadratics through the cusps:

In [150]:
matrix([[m(list(c)) for c in XNS13_cusps] for m in mons2]).nullity()

0

so we look for cubics.  As there are only $6$ cusps now, the space of
cubics through them has dimension $10-6=4$:

In [151]:
ker = matrix([[m(list(c)) for c in XNS13_cusps] for m in mons3]).left_kernel()
ker.dimension()

4

We find a small $\Z$-basis as before:

In [152]:
M = ker.basis_matrix()
M = (M*M.denominator()).change_ring(ZZ)
Sm,U,V = M.smith_form()
M = (Sm.submatrix(ncols=4)^(-1)*U*M).change_ring(ZZ)
M = (M*M.transpose()).LLL_gram().transpose() * M
cubs = [sum([c*m for c,m in zip(r,mons3)]) for r in M.rows()]
cub = cubs[0]

The first cubic is

In [153]:
cub

-31*X^3 + 109*X^2*Y + 34*X*Y^2 - 50*Y^3 - 71*X^2*Z + 54*X*Y*Z + 63*Y^2*Z - 70*X*Z^2 - 91*Y*Z^2 + 27*Z^3

which will serve as it does not pass through any of the rational
points:

In [154]:
print([cub(list(p)) for p in PtsQ])
assert all([cub(list(p))!=0 for p in XNS13Q])
assert all([cub(list(p))==0 for p in XNS13_cusps])

[27, -50, 27, -26]


To ease the linear algebra it is worthwhile to convert the cusp forms
to lie in $\Q(\zeta_{13}^{+})[[q]]$ instead of $\Q(\zeta_{13})[[q]]$.
We have not used this field before so need to set this up:

In [155]:
z13p = zeta13+zeta13^12 # generates sextic subfield
Q13p.<z13p> = NumberField(z13p.minpoly(), embedding=z13p)
eQ13pQ13 = Q13p.hom([Q13(z13p)])
Q13pq.<q> = PowerSeriesRing(Q13p,ps_prec)

Now we do the conversion, dividing each $q$-expansion by $q$ at the
same time:

In [156]:
ffq = [Q13pq([eQ13pQ13.preimage(c) for c in list(f)[1:]]).add_bigoh(ps_prec-1)
                  for f in NS_forms]

Compute the denominator, the $q$-expansion of $j(q^{13})$ in the
sextic field, and their product:

In [157]:
denj = cub(ffq)
denj13 = denj^13
jq = j_invariant_qexp(20, Q13p)
q = jq.parent().gen()
jq13 = jq(q^13)
numjq39 = jq13*denj13

Compute the degree $39$ monomials:

In [158]:
xq,yq,zq = ffq
xpowers = [xq.parent()(1)]
ypowers = [yq.parent()(1)]
zpowers = [zq.parent()(1)]
while len(xpowers)<40:
    xpowers += [xq*xpowers[-1]]
    ypowers += [yq*ypowers[-1]]
    zpowers += [zq*zpowers[-1]]

mons39q = [xpowers[m.degrees()[0]]*ypowers[m.degrees()[1]]*zpowers[m.degrees()[2]] for m in mons39r]

Set up and solve the linear equations over $\Q$:

In [159]:
arr = [[-numjq39[i] for i in range(160)]] \
       + [[mq[i] for i in range(160)] for mq in mons39q]
relmatQ =  Matrix([  flatten([list(aij) for aij in ai])  for ai in arr])
ker = relmatQ.left_kernel()
ker.dimension()

1

The solution space has dimension $1$ and we
extract the coefficient vector:

In [160]:
b = ker.basis()[0]
b *= b.denominator()
jnum = sum([bi*m for bi,m in zip(list(b)[1:],mons39r)])
jden = b[0]*cub^13

These are the numerator and denominator of the $j$-map, each expressed
as a polynomial of degree $39$ in $\Q[X,Y,Z]$.

### $j$-invariants of rational points

Now we evaluate the map at the rational points:

In [161]:
jlist = [QQ(jnum(list(p))/jden(list(p))) for p in XNS13Q]
jlist

[8000, -32768, -3375, -147197952000, -884736, 16581375]

It appears that these $j$-invariants are all CM, which we check:

In [162]:
all([j in cm_j_invariants(QQ) for j in jlist])

True

Finally we check that the corresponding orders agree with Burcu
Baran's Table 1.1:

In [163]:
CMQ = cm_j_invariants_and_orders(QQ)
[(P,d*f^2) for P,jj in zip(XNS13Q,jlist) for (d,f,j) in CMQ if j==jj]

[((-17 : -9 : 1), -8),
 ((-10/9 : -1/3 : 1), -11),
 ((-8/15 : 8/15 : 1), -7),
 ((9/14 : 17/14 : 1), -67),
 ((7/8 : 3/4 : 1), -19),
 ((2 : 2 : 1), -28)]